# Check GPUs and Install libraries

In [1]:
#@title 1.1 Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
  #!nvidia-smi
  nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
else:
  #!nvidia-smi -i 0 -e 0
  nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_output)
  nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  print(nvidiasmi_ecc_note)

GPU 0: NVIDIA L4 (UUID: GPU-417126cd-a631-a8e0-0982-caf3b63f21a4)



# Imports

In [2]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
from datasets import Dataset as HFDataset

from peft import LoraConfig, get_peft_model
from datasets import load_dataset

from transformers import (
    pipeline,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertConfig,
    BertModel,
    Trainer,
    TrainingArguments,
    T5Tokenizer,
    T5ForConditionalGeneration,
    AutoTokenizer, 
    AutoModelForCausalLM,
    GPT2Model,
    GPT2Config,
    GPT2ForSequenceClassification,
    GPT2Tokenizer,
)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Loading and Data pre-processing

In [3]:
train_hf_dataset = HFDataset.load_from_disk("SubBigCloneBench_train")
validation_hf_dataset = HFDataset.load_from_disk("SubBigCloneBench_validation")
test_hf_dataset = HFDataset.load_from_disk("SubBigCloneBench_test")
print(train_hf_dataset)
print(validation_hf_dataset)
print(test_hf_dataset)

Dataset({
    features: ['label', 'func1', 'func2', 'id', '__index_level_0__'],
    num_rows: 2000
})
Dataset({
    features: ['label', 'func1', 'func2', 'id', '__index_level_0__'],
    num_rows: 1999
})
Dataset({
    features: ['label', 'func1', 'func2', 'id', '__index_level_0__'],
    num_rows: 999
})


In [4]:
# Pre-process the original data
def preprocess_function(examples):
    inputs = ["code_1: " + examples["func1"][i] + " code_2: " + examples["func2"][i] for i in range(len(examples["func1"]))]
    targets = [[i] for i in examples["label"]]
    # targets = [str(i) for i in examples["label"]]
    
    max_length = 512
    return {"input_ids": tokenizer(inputs, truncation=True, padding="max_length", max_length=max_length)["input_ids"],
            "attention_mask": tokenizer(inputs, truncation=True, padding="max_length", max_length=max_length)["attention_mask"],
            "labels": torch.tensor(targets, dtype=torch.long),
            # "labels": tokenizer(targets, truncation=True, padding="max_length", max_length=256)["input_ids"]
        }

configuration = GPT2Config()

# Load the Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Or assign a suitable token

# Pre-process data
tokenized_train_dataset = train_hf_dataset.map(preprocess_function, batched=True, batch_size=4)
tokenized_validation_dataset = validation_hf_dataset.map(preprocess_function, batched=True, batch_size=4)
tokenized_test_dataset = test_hf_dataset.map(preprocess_function, batched=True, batch_size=4)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

In [6]:
# Load pre-trained model
num_labels = 2
model = GPT2ForSequenceClassification(configuration).from_pretrained("gpt2", num_labels=num_labels)
model.config.pad_token_id = model.config.eos_token_id
# Config LoRA specifications
lora_config = LoraConfig(
    # r=8,  # lower the rank
    r=48,  # lower the rank
    lora_alpha=32,
    # target_modules=["q", "v"],  # apply LoRA to q and v of attention modules
    target_modules=["c_attn", "attn.c_attn"],  # target query, key, and value together
    lora_dropout=0.1,
    bias="none",
    # task_type="SEQ_2_SEQ_LM",  # task type set to seq2seq generation
    task_type="SEQ_CLS",  # task type set to text classification
    fan_in_fan_out=True,
)

# Convert the model to LoRA model
model = get_peft_model(model, lora_config)

# Check the number of trainable parameters (for LoRA)
model.print_trainable_parameters()

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,771,008 || all params: 126,212,352 || trainable%: 1.4032


In [7]:
# Adjust the batch size
batch_size = 4
training_args = TrainingArguments(
    output_dir="./results/lora-GPT2-ccd",
    eval_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    metric_for_best_model="f1",
    weight_decay=0.01,
    num_train_epochs=6,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    report_to="mlflow",  # disable wandb etc.
    fp16=True,  # mixed precision training
    optim="adamw_torch",  # use torch original optimizer
)

def compute_metrics(pred):
    labels = pred.label_ids
    # Extract logits from the predictions & assuming logits are the first element in the tuple
    logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = logits.argmax(-1)
    f1 = f1_score(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
)

# Start Training
trainer.train()

# Save the model after LoRA fine-tuing
model.save_pretrained("./lora-GPT2-ccd/model")
tokenizer.save_pretrained("./lora-GPT2-ccd/tokenizer")

print("LoRA fine-tuning done, model saved!")

test_results = trainer.predict(tokenized_test_dataset)
predictions, labels, metrics = test_results
print(metrics)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.580500,0.334626,0.850925,0.688408,0.735265,0.707013
2,0.552100,0.923150,0.755378,0.647673,0.790471,0.655591
3,0.310100,0.387037,0.864432,0.727090,0.839969,0.762389
4,0.295100,0.474780,0.869435,0.732374,0.839556,0.767395
5,0.144000,0.382790,0.895448,0.769073,0.864358,0.804360
6,0.176900,0.466884,0.885443,0.756444,0.873399,0.795430


LoRA fine-tuning done, model saved!


{'test_loss': 0.5378929376602173, 'test_accuracy': 0.8768768768768769, 'test_precision': 0.7521551724137931, 'test_recall': 0.8611233424221383, 'test_f1': 0.7883930185901133, 'test_runtime': 8.4489, 'test_samples_per_second': 118.24, 'test_steps_per_second': 29.59}
